# Import packages and load data

In [1]:
import pandas as pd
import numpy as np
import os
from itertools import product
import matplotlib.pyplot as plt
import time
import gc
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn import preprocessing
import xgboost
import lightgbm as lgb


%matplotlib inline 


In [2]:
DATA_FOLDER = '../readonly/final_project_data/'

transactions    = pd.read_csv(os.path.join(DATA_FOLDER, 'sales_train.csv.gz'))
items           = pd.read_csv(os.path.join(DATA_FOLDER, 'items.csv'))
item_categories = pd.read_csv(os.path.join(DATA_FOLDER, 'item_categories.csv'))
shops           = pd.read_csv(os.path.join(DATA_FOLDER, 'shops.csv'))

test_set=pd.read_csv(os.path.join(DATA_FOLDER, 'test.csv.gz'))

# Dataset exploration 
based on Pandas basics notebook from week1

In [3]:
## From week 1:
# How many items are there, such that their price stays constant during the whole period of time?
# Remember, the data can sometimes be noisy. :)
# grouping3=pd.DataFrame(transactions[['item_id','item_price']],copy=True)
# grouping3['item_price']=abs(grouping3['item_price']).round(decimals=2)
# grouping3=grouping3.drop_duplicates()
# sum3=grouping3.groupby('item_id').count()
# sum3[sum3['item_price']==1].count()

# Maybe could be the feature (price changes count, last price delta, price changed time)

## EDA - to do :
 1. analyze time dependancies if any for different intervals
 2. plot some dependancies?
 3. From the task: Target distribution is visualized, time trend is assessed

# Preparing simple submittion 

<b>A good exercise is to reproduce previous_value_benchmark.</b>

<i> Stolen from somewhere from course materials </i>


As the name suggest - in this benchmark for the each shop/item pair our predictions are just monthly sales from the previous month, i.e. October 2015.

The most important step at reproducing this score is correctly aggregating daily data and constructing monthly sales data frame. You need to get lagged values, fill NaNs with zeros and clip the values into [0,20] range. If you do it correctly, you'll get precisely 1.16777 on the public leaderboard.

In [4]:
# df_filter=transactions.set_index(pd.to_datetime(transactions["date"],format='%d.%m.%Y'))["2015-10"]
# df_october=pd.DataFrame(df_filter)

# df_october_groupped = df_october.groupby(['shop_id','item_id'],as_index=False).agg({'item_cnt_day':'sum'})

# predictions = pd.merge(test_set,df_october_groupped,
#                        left_on=('shop_id','item_id'), right_on=('shop_id','item_id'), how='left')
# ##filling NAs
# predictions.item_cnt_day=predictions.item_cnt_day.fillna(0.0)
# ## clipping
# predictions.loc[predictions.item_cnt_day>20,'item_cnt_day']=20
# predictions.loc[predictions.item_cnt_day<0,'item_cnt_day']=0
# ##dropping extra colums and indexes after merge
# predictions.drop(labels=['shop_id','item_id'],axis=1,inplace=True)
# predictions.rename(columns={'item_cnt_day':'item_cnt_month'},inplace=True)

# # converting  ID back to int
# predictions.ID = predictions.ID.astype(int)

# filename = 'submition_{}.csv'.format(time.strftime('%Y_%m_%d_%H_%M',time.localtime()))
# filename = os.path.join(DATA_FOLDER,filename)
# predictions.to_csv(filename,index=False)

# Train test split investigation


In [5]:
items_test = test_set.item_id.nunique()
shops_test = test_set.shop_id.nunique()

items_train = transactions.item_id.unique().shape[0]
shops_train = transactions.shop_id.unique().shape[0]

print ('Test set: \n'+\
       ' items: ' + str(items_test) + ' shops: ' + str(shops_test) + \
       '\n total: '+ str(test_set["ID"].count()) + ' supposed total: '+str(items_test*shops_test) + \
       '\nData set: \n'+\
       ' items: ' + str(items_train) + ' shops: ' + str(shops_train) + \
       '\n total: ' + str (items_train*shops_train)
      )



Test set: 
 items: 5100 shops: 42
 total: 214200 supposed total: 214200
Data set: 
 items: 21807 shops: 60
 total: 1308420


We can see the cartesian product is used in a test set, however only some of shops and items included into test set.  This suggests potential dataleak, but further investigation needed.
also hypotesis to test - shops are beeing closed, as well items are get offsale. Maybe data missing in testset is just one which are not exists anymore

In [6]:
excluded_shops = np.setxor1d(transactions.shop_id.unique(),test_set.shop_id.unique())
items_train_only = transactions[~transactions.item_id.isin(test_set.item_id)].item_id.unique()

print ('excluded_shops ' + str(excluded_shops))
print ('items only in train count ' + str(items_train_only.shape[0]))

# There is also items only in test set, but let's ignore it for now
print ('items only in test count ' + str(test_set[~test_set.item_id.isin(transactions.item_id.unique())].\
                                         item_id.unique().shape[0]))
print ('average date_block for: \n'+\
       '  exluded shops: ' + \
        str(transactions[transactions.shop_id.isin(excluded_shops)].date_block_num.mean()) + \
       '\n  excluded items: ' + \
        str(transactions[transactions.item_id.isin(items_train_only)].date_block_num.mean()) + \
        '\n  average for all data : ' + \
        str(transactions.date_block_num.mean()) + \
       '\nlast seen:' + \
       '\n  exluded shops: ' + \
       str(transactions[transactions.shop_id.isin(excluded_shops)]. \
           groupby('shop_id'). \
           agg({'date_block_num':'max'}).mean().values[0]) +\
       '\n  excluded items: ' + \
       str(transactions[transactions.item_id.isin(items_train_only)]. \
           groupby('item_id'). \
           agg({'date_block_num':'max'}).mean().values[0]) +\
       '\n  average for all shops : ' + \
       str(transactions.groupby('shop_id').agg({'date_block_num':'max'}).mean().values[0]) +\
       '\n  average for all items : ' + \
       str(transactions.groupby('item_id').agg({'date_block_num':'max'}).mean().values[0]) 
      )




excluded_shops [ 0  1  8  9 11 13 17 20 23 27 29 30 32 33 40 43 51 54]
items only in train count 17070
items only in test count 363
average date_block for: 
  exluded shops: 11.962617895419658
  excluded items: 10.468482249471924
  average for all data : 14.56991146343017
last seen:
  exluded shops: 20.38888888888889
  excluded items: 18.841769185705918
  average for all shops : 29.216666666666665
  average for all items : 21.760489750997387


seems average date is less for absent shops, so we can somehow conlude the shops not in test set are closed. 
However, it's not that clear for items, why only about 1/4 of items are in test set. 

It also doesn't looks like there are only items which was sold, as leaderboard probing shown mean 0.2839
https://www.kaggle.com/c/competitive-data-science-predict-future-sales/discussion/79142

There could be a case items excluded are the one's had 0 sales in all the possible shops within the given month (which also suggested by train/test split in week4). The last one can be tested by grouping transactions by date_block_num and count unique items over to see if it's close to test set size

This can be utilized together with mean information above during predicting for items not in dataset.

In [7]:
transactions.groupby('date_block_num').agg({'item_id':'nunique'}).mean().values[0]

## Seems pretty close (espesially if look and number's not mean - 5100 looks reasonable values)

6879.764705882353

# Validation

Before playing with any kind of models validation has to be setup. 
Some ideas could be taken from week4. 

In contrast with it 4 cycles of validation to be used - train on date_block_num [0:i] -> validate on [i+1], where i [30,31,32,33]



Let's assume every model should take dataset like transations and test pairs (like test_set) and return dataset of prediction 
[shop_id, item_id, prediction] for the month, next to the last present in dataset. 



In [8]:
def constant_model(train_set, test_set,fit_col):
    # we can take global mean of training data, but it's not so easy to calculate, 
    # as we need to produce proper fraction of zeros. So just constant for some period taken
    return np.ones((test_set.shape[0],1),dtype='float32')*0.298 ## true_values['item_cnt_day'].mean()

In [9]:
def previous_value_model(train_set, test_set,fit_col):
    max_date_block = train_set.date_block_num.max()
    
    predictions = test_set.merge(train_set[train_set.date_block_num == max_date_block],\
        how='left',on=['shop_id','item_id']).fillna(0.0)
    
    predictions.loc[predictions.target>20,'target']=20
    predictions.loc[predictions.target<0,'target']=0
    
    return predictions['target'].values

In [10]:
def lin_model (train_set, test_set,fit_col):
    lr = LinearRegression()
    X = train_set[fit_col]
    y = train_set['target']
    lr.fit(X.values,y.values)
    predictions = lr.predict(test_set[fit_col].values)

    return predictions.clip(min=0.0, max =20.0)


In [11]:
def lin_model_with_feature_norm (train_set, test_set,fit_col):
    ## Seems with MinMaxScaler exactly the same metrics as lin_model. Just, probably given by the 
    ## fact feature are centered in LinearRegression and not too far from 0-1
    ## 

    #     scaler = preprocessing.MinMaxScaler()
    #     scaler = preprocessing.StandardScaler()
    scaler = preprocessing.RobustScaler()

    reduced_X_train = pd.DataFrame(train_set[fit_col+['target']],copy=True)
    X_train_minmax = scaler.fit_transform(reduced_X_train[fit_col])
    reduced_X_train[fit_col] = X_train_minmax

    reduced_X_test = pd.DataFrame(test_set[fit_col],copy=True)
    X_test_minmax = scaler.transform(reduced_X_test)
    reduced_X_test[fit_col] = X_test_minmax

    predictions=lin_model(reduced_X_train,reduced_X_test,fit_col)
    
    del reduced_X_train, reduced_X_test, X_train_minmax, X_test_minmax
    gc.collect();

    return predictions

In [12]:
def xgboost_model(train_set, test_set,fit_col):
    xgb_model = xgboost.XGBRegressor(n_jobs=4,objective='reg:squarederror')
    xgb_model.fit(train_set[fit_col].values,train_set['target'].values)
    predictions = xgb_model.predict(test_set[fit_col].values)
    
    return predictions.clip(min=0.0, max =20.0)

In [13]:
def lightgbm_model(train_set, test_set,fit_col):
    lgb_params = {
#                'feature_fraction': 0.75,
               'metric': 'rmse',
               'nthread':4, 
#                'min_data_in_leaf': 2**7, 
#                'bagging_fraction': 0.75, 
#                'learning_rate': 0.03, 
               'objective': 'mse', 
#                'bagging_seed': 2**7, 
#                'num_leaves': 2**7,
#                'bagging_freq':1,
#                'verbose':0 
              }

    model = lgb.train(lgb_params, lgb.Dataset(train_set[fit_col].values, label=train_set['target'].values), 100)
    predictions = model.predict(test_set[fit_col].values)
    return predictions.clip(min=0.0, max =20.0)

In [14]:
def cartesian_product(*arrays):
    la = len(arrays)
    dtype = np.result_type(*arrays)
    arr = np.empty([len(a) for a in arrays] + [la], dtype=dtype)
    for i, a in enumerate(np.ix_(*arrays)):
        arr[...,i] = a
    return arr.reshape(-1, la)

In [15]:
def generate_shifted_features(trainset, testset):
    
    '''
    added features for sale in same shop, then for same item and then for pair (shop, item) for 1,2,3,4,6,9,12 month ago
    in : trainset - pandas dataframe with train data (transactions), testset - shop_id, item_id pairs for predictions
    out - tuple (trainset, testset), where trainset is groupped by shop_id, item_id either
    
    '''
    index_cols = ['shop_id', 'item_id', 'date_block_num']
    shift_range = [1, 2, 3, 4, 5, 12]
    # shift_range = [1, 2]

    grid = [] 
    for block_num in trainset['date_block_num'].unique():
        cur_shops = trainset.loc[trainset['date_block_num'] == block_num, 'shop_id'].unique()
        cur_items = trainset.loc[trainset['date_block_num'] == block_num, 'item_id'].unique()
        # https://docs.python.org/3/library/itertools.html#itertools.product
        # Cartesian product of input iterables.
        grid.append(cartesian_product(cur_shops, cur_items, np.array([block_num])))
    # Turn the grid into a dataframe
    grid = pd.DataFrame(np.vstack(grid), columns = index_cols,dtype=np.int32)
    gb = trainset.groupby(index_cols,as_index=False).agg({'item_cnt_day':'sum'})
    gb.rename(columns={'item_cnt_day':'target'},inplace=True)
    # Join it to the grid
    all_data = pd.merge(grid, gb, how='left', on=['shop_id','item_id','date_block_num']).fillna(0)

    # Same as above but with shop-month aggregates
    gb = trainset.groupby(['shop_id', 'date_block_num'],as_index=False).agg({'item_cnt_day':'sum'})
    gb.rename(columns={'item_cnt_day':'target_shop'},inplace=True)
    all_data = pd.merge(all_data, gb, how='left', on=['shop_id', 'date_block_num']).fillna(0)

    # Same as above but with item-month aggregates
    gb = trainset.groupby(['item_id', 'date_block_num'],as_index=False).agg({'item_cnt_day':'sum'})
    gb.rename(columns={'item_cnt_day':'target_item'},inplace=True)
    all_data = pd.merge(all_data, gb, how='left', on=['item_id', 'date_block_num']).fillna(0)
    
    # List of columns that we will use to create lags
    cols_to_rename = list(all_data.columns.difference(index_cols))
    
    max_block_num = all_data.date_block_num.max()
    testset['date_block_num']=max_block_num+1
    # shifted features are also done in kinda weird fashion - probably there is easier way 
    for month_shift in shift_range:
        train_shift = all_data[index_cols + cols_to_rename].copy()
        train_shift['date_block_num'] = train_shift['date_block_num'] + month_shift
        foo = lambda x: '{}_lag_{}'.format(x, month_shift) if x in cols_to_rename else x
        train_shift = train_shift.rename(columns=foo) #rename using lambda.Doesn't seems most transparent, but ok
        train_shift.head()        
        testset = pd.merge(testset, train_shift, on=index_cols, how='left').fillna(0)
        all_data = pd.merge(all_data, train_shift, on=index_cols, how='left').fillna(0)
        
    del train_shift

    # Don't use old data which have 0 shift and will just confuse the model
    all_data = all_data[all_data['date_block_num'] >= 12]

    # List of all lagged features
    # this will cause a problem, if 1 is excluded and 10 included, for ex. But probably this shouldn't happen
    fit_cols = [col for col in all_data.columns if col[-1] in [str(item) for item in shift_range]] 
    # We will drop these at fitting stage
    to_drop_cols = list(set(list(all_data.columns)) - (set(fit_cols)|set(index_cols)) - set (['target'])) 
    all_data.drop(to_drop_cols,axis=1,inplace=True)
    
    
    del grid, gb 
    gc.collect();
    
    return (all_data,testset,fit_cols)


In [16]:
from_block = 30
to_block = 33

def validate (models):
    outcome = []
    for i in range(from_block,to_block+1):    
        print ('validation: generating test and train for i='+str(i))
        train_transactions = transactions[transactions.date_block_num<i]
        test_transactions = transactions[transactions.date_block_num==i]

        test_set = pd.DataFrame(cartesian_product(test_transactions.shop_id.unique(), 
                                       test_transactions.item_id.unique()),columns=['shop_id','item_id'])

        true_values = test_set.merge ( \
            test_transactions.groupby(['shop_id','item_id'],as_index=False).agg({'item_cnt_day':'sum'}), \
            how = 'left' ).fillna(0)
        


        (train_groupped_with_shifted, test_set_with_shifted, fit_col) = \
            generate_shifted_features(train_transactions,true_values[['shop_id','item_id']])

        for model in models:
            print ('validation: testing model ' + model.__name__)
            predictions = model(train_groupped_with_shifted, test_set_with_shifted,fit_col)
            
            # According to competition description validation is done with clipping, so we should do the same
            # Model is responsible for it's own clipping if nessesary
            # r2 should be as close to 1 as possible. 0 is constant model            
            r2s = r2_score(true_values['item_cnt_day'].clip(lower=0.0, upper =20.0), predictions)
            rmse = np.sqrt(mean_squared_error(true_values['item_cnt_day'].clip(lower=0.0, upper =20.0), predictions))
            
            outcome.append({'model':model.__name__,'block':i,'metric':'r2s','value':r2s})
            outcome.append({'model':model.__name__,'block':i,'metric':'rmse','value':rmse})
                
        print () 
        
    return outcome

In [17]:

# resut = validate([constant_model,previous_value_model,lin_model,xgboost_model])
resut = validate([constant_model,previous_value_model,lin_model, lin_model_with_feature_norm, lightgbm_model])

pd.DataFrame(resut).groupby(['model','metric'],as_index=False).agg({'value':'mean'})

validation: generating test and train for i=30
validation: testing model constant_model
validation: testing model previous_value_model
validation: testing model lin_model
validation: testing model lin_model_with_feature_norm
validation: testing model lightgbm_model

validation: generating test and train for i=31
validation: testing model constant_model
validation: testing model previous_value_model
validation: testing model lin_model
validation: testing model lin_model_with_feature_norm
validation: testing model lightgbm_model

validation: generating test and train for i=32
validation: testing model constant_model
validation: testing model previous_value_model
validation: testing model lin_model
validation: testing model lin_model_with_feature_norm
validation: testing model lightgbm_model

validation: generating test and train for i=33
validation: testing model constant_model
validation: testing model previous_value_model
validation: testing model lin_model
validation: testing model li

,model,metric,value
0,constant_model,r2s,-0.000926
1,constant_model,rmse,1.086481
2,lightgbm_model,r2s,0.370840
3,lightgbm_model,rmse,0.862635
4,lin_model,r2s,0.307244
5,lin_model,rmse,0.905361
6,lin_model_with_feature_norm,r2s,0.307244
7,lin_model_with_feature_norm,rmse,0.905361
8,previous_value_model,r2s,0.138526
9,previous_value_model,rmse,1.008575


Validation:

model 	metric 	value
0 	constant_model 	r2s 	-0.000926
1 	constant_model 	rmse 	1.086481
2 	lin_model 	r2s 	0.307244
3 	lin_model 	rmse 	0.905361
4 	previous_value_model 	r2s 	0.138526
5 	previous_value_model 	rmse 	1.008575
6 	xgboost_model 	r2s 	0.358721
7 	xgboost_model 	rmse 	0.870930


<b> Seems local validation somehow corelates with public leaderboard </b>

|Name|Score|Local score|Model|Comment|
|----|-----|-----------|-----|-------|
|submition_2019_06_03_09_29.csv.gz|1.16777|1.008575|Last month baseline||
|submition_lin_model_2019_12_02_13_50.csv.gz|1.96182||simple linear model with 2 month shifts and with clipping 0-300|wrong cliipng|
|submition_lin_model_2019_12_02_13_56.csv.gz|4.34576||simple linear model with 2 month shifts with clipping removed|no clipping|
|submition_lin_model_2019_12_02_15_39.csv.gz|1.08014|0.905361|simple linear model, more with proper clipping|proper clipping apllied|
|submition_xgboost_model_2019_12_03_15_43.csv.gz|1.02016|0.870930|simple xgboost without any parameters tuned|
|submition_lightgbm_model_2019_12_04_05_53.csv.gz|**1.00715**|0.862635|simple light gbm without any parameters tuned|



In [18]:
def prepare_submit(model):
    
    (train_groupped_with_shifted, test_set_with_shifted, fit_col) = \
            generate_shifted_features(transactions,test_set)
    
    predictions = model(train_groupped_with_shifted, test_set_with_shifted,fit_col)
    test_set_with_shifted ['predictions'] = predictions
    
    # adjust types
    test_set_with_shifted.ID = test_set_with_shifted.ID.astype(int)
    test_set_with_shifted.rename(columns={'predictions':'item_cnt_month'},inplace=True)

    filename = 'submition_{}_{}.csv.gz'.format(model.__name__,time.strftime('%Y_%m_%d_%H_%M',time.localtime()))
    test_set_with_shifted[['ID','item_cnt_month']].to_csv(filename,index=False)
    print (filename + ' prepared')
    
    
prepare_submit(lightgbm_model)


submition_lightgbm_model_2019_12_05_06_36.csv.gz prepared


### Validation: TODO
1. From the task - Type of public/private split is identified
2. Compare for more complex model which metric better correlates with lederboard - rmse or r2

In [19]:
# # # out of functions code to test model

# train_transactions = transactions[transactions.date_block_num<32]
# test_transactions = transactions[transactions.date_block_num==33]

# test_set = pd.DataFrame(cartesian_product(test_transactions.shop_id.unique(), 
#                                test_transactions.item_id.unique()),columns=['shop_id','item_id'])

# true_values = test_set.merge ( \
#     test_transactions.groupby(['shop_id','item_id'],as_index=False).agg({'item_cnt_day':'sum'}), \
#     how = 'left' ).fillna(0)

# (train_groupped_with_shifted, test_set_with_shifted, fit_col) = \
#     generate_shifted_features(train_transactions,true_values[['shop_id','item_id']])

# train_set = train_groupped_with_shifted
# test_set = test_set_with_shifted

# # predictions = lin_model(train_groupped_with_shifted, test_set_with_shifted,fit_col)
# # rmse = np.sqrt(mean_squared_error(true_values['item_cnt_day'].clip(lower=0.0, upper =20.0).values, predictions))


In [20]:
# # 
# xgb_model = xgboost.XGBRegressor(n_jobs=4,objective='reg:squarederror',verbosity=2)
# xgb_model.fit(train_groupped_with_shifted[fit_col],train_groupped_with_shifted['target'])
# predictions = xgb_model.predict(test_set_with_shifted[fit_col])
# rmse = np.sqrt(mean_squared_error(true_values['item_cnt_day'].clip(lower=0.0, upper =20.0).values, predictions.clip(min=0.0, max =20.0)))

# Feature engineering

1. First let's do obvious thing suggested in week4 - adding as features 1-2-3-4 month back sales, pass if over model and see what will happen to metrics, local validation and submitted score

